# Darwin Dataset
#This notebook will look to calling national rail Darwin API to download and extract Darwin Timetable Data

In [5]:
#Import Packages
import boto3
import os
import re
import pandas as pd
import gzip
import xmltodict

In [2]:
s3 = boto3.resource(
    service_name = 's3',
    region_name='eu-west-1',
    aws_access_key_id = 'SECRET',
    aws_secret_access_key = 'SECRET'
    )

In [3]:
#set the bucket

#Check if directory exists

directory  = "C:/Users/Mohammed/myFolder/Output"
    
#os.mkdir("C:/Users/Mohammed/myFolder/Output")


bucket = s3.Bucket('darwin.xmltimetable')
#print the objects within the bucket
for obj in bucket.objects.all():
    if re.search("_v8.xml.gz", obj.key):
        temp_string = str(obj.key).split("/")[1]
        #print(temp_string)
        s3.Bucket('darwin.xmltimetable').download_file(obj.key, Filename = directory + "/" + temp_string)
        print(obj.key)

PPTimetable/20220523023948_v8.xml.gz
PPTimetable/20220524023843_v8.xml.gz
PPTimetable/20220525023831_v8.xml.gz
PPTimetable/20220526023801_v8.xml.gz
PPTimetable/20220527023502_v8.xml.gz
PPTimetable/20220528022757_v8.xml.gz
PPTimetable/20220529022938_v8.xml.gz
PPTimetable/20220530023632_v8.xml.gz


In [6]:
temp_array = []
filepath = "C:/Users/Mohammed/myFolder/Output/20220522022909_v8.xml.gz"

def extract_and_append(filepath):
    with gzip.open(filepath, 'rb') as f:
        file_content = f.read()
    
    #open filecontent xml
    d = xmltodict.parse(file_content)
    
    #read xml into a dataframe.
    #read only the Journey Node from the XML document
    df = pd.DataFrame.from_dict(d['PportTimetable']['Journey'])
    
    #iterate through the dataframe relevant columns and extract the values into a temp array
    cols = ["OR", "IP", "PP", "DT"]
    status = ["User", "User", "Passing Point", "User"]

    for item in zip(cols, status):
        for index, row in df.iterrows():
            #print(row["@rid"])
            TOC = row["@toc"]
            #print(row["IP"])
            if type(row[item[0]]) == list:
                for i in range(len(row[item[0]])):
                    pp_tiploc = row[item[0]][i]["@tpl"]
                    temp_array.append([TOC, item[1], pp_tiploc])

                    
                    
print(len(temp_array))
        

0


In [12]:
#iterate through a directory and call the function to append_temparray

import glob
#columns to iterate through

for filename in glob.iglob(f'{directory}/*.gz'):
    print(filename)
    extract_and_append(filename)



print(len(temp_array))    

C:/Users/Mohammed/myFolder/Output\20220522022909_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220523023948_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220524023843_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220525023831_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220526023801_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220527023502_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220528022757_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220529022938_v8.xml.gz
C:/Users/Mohammed/myFolder/Output\20220530023632_v8.xml.gz
7369127


In [86]:
#convert the temp_array into a dataframe
station_tocs_original = pd.DataFrame(temp_array, columns = ["TOC","User/Pass", "TIPLOC"])

#Reduce the list by removing duplicate TOCs from Stations
station_tocs_reduced = station_tocs_original.groupby(["TOC", "TIPLOC","User/Pass"])["TOC"].count().reset_index(name="count")

station_tocs_reduced

station_tocs_reduced.to_csv(directory + "/" + "station_tocs_timetable_output.csv")
station_tocs_reduced

,TOC,TIPLOC,User/Pass,count
0,AW,ABCWM,Passing Point,924
1,AW,ABDVY,User,330
2,AW,ABER,User,1613
3,AW,ABGLELE,Passing Point,400
4,AW,ABGLELE,User,873
...,...,...,...,...
10395,ZZ,WYLYAHB,Passing Point,28
10396,ZZ,YATE,Passing Point,6
10397,ZZ,YORK,Passing Point,2
10398,ZZ,YORK,User,2


## TOC Station Probabilities
This section will look to see the probability of a TOC pulling into a station based on the number of times it has attended/departed from a particular station. 

In [155]:
#Group by Station

#Look at users of the station only
Station_Tocs_Users = station_tocs_reduced[station_tocs_reduced["User/Pass"]=="User"]
Station_Tocs_Users

#Get total TOC visits to each station
station_total= Station_Tocs_Users.groupby(["TIPLOC"])["count"].sum().reset_index(name = "Total Visits")
station_total

#work out the ratio of visit by overall visit
Station_toc_users_total = Station_Tocs_Users.merge(station_total, how="left", on="TIPLOC")
#station_probabilities
Station_toc_users_total["Frequency %"] = round((Station_toc_users_total["count"] / Station_toc_users_total["Total Visits"])*100,2)
Station_toc_users_total["TOC_New"] = Station_toc_users_total["TOC"] + " (" + Station_toc_users_total["Frequency %"].astype(str) + "%)"

#output the file
station_toc_probabilities_Final = Station_toc_users_total.groupby('TIPLOC')['TOC_New'].apply('; '.join).reset_index()
station_toc_probabilities_Final.to_csv("Station_TOC_Probabilities_final_Users.csv")

In [156]:
#Look at passing point

#Look at users of the station only
Station_Tocs_PassingPoint = station_tocs_reduced[station_tocs_reduced["User/Pass"]=="Passing Point"]
station_total_passingPoint= Station_Tocs_PassingPoint.groupby(["TIPLOC"])["count"].sum().reset_index(name = "Total Visits")
station_total_passingPoint

#Get total TOC visits to each station
Station_toc_users_total_pp = Station_Tocs_PassingPoint.merge(station_total_passingPoint, how="left", on="TIPLOC")
Station_toc_users_total_pp

#work out the ratio of visit by overall visit
#station_probabilities
Station_toc_users_total_pp["Frequency %"] = round((Station_toc_users_total_pp["count"] / Station_toc_users_total_pp["Total Visits"])*100,2)
Station_toc_users_total_pp["TOC_New"] = Station_toc_users_total_pp["TOC"] + " (" + Station_toc_users_total_pp["Frequency %"].astype(str) + "%)"

#output the file
station_toc_probabilities_Final_pp = Station_toc_users_total_pp.groupby('TIPLOC')['TOC_New'].apply('; '.join).reset_index()
station_toc_probabilities_Final_pp.to_csv("Station_TOC_Probabilities_final_PassingPoint.csv")

In [158]:
#Additional Combine the users and passing points together
station_toc_probabilities_Final2 = station_toc_probabilities_Final.merge(station_toc_probabilities_Final_pp, how="left", on="TIPLOC")
station_toc_probabilities_Final2.to_csv("Station_TOC_Probabilities_combined.csv")